# Data Setup
Configuring CosmosDB

In [23]:
!python initialize_cosmos_db.py

Inserting 5 sample KYC records...
Upserted record for Satya Narayana Nadella
Upserted record for Mustafa Suleyman
Upserted record for Samuel Harris Altman
Upserted record for Alain Berset
Upserted record for Sal Khan
Done.


# Environment Setup
Import required libraries and load environment variables using python-dotenv.

In [ ]:
!pip install -r requirements.txt

In [25]:
# Import required libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

True

# Azure AI Client Configuration
Configure Azure AI client using DefaultAzureCredential and connection string.

In [26]:
# Import Azure identity and AI project client libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Initialize Azure AI client using DefaultAzureCredential and connection string from environment variables
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

# Bing Search Integration
Set up Bing search tool using BingGroundingTool and handle connection configuration.

In [27]:
from azure.ai.projects.models import BingGroundingTool, FunctionTool, ToolSet

# Initialize Bing tool as None
bing_tool = None

# Get Bing connection name from environment variables
bing_connection_name = os.environ.get("BING_CONNECTION_NAME")

# If Bing connection name is available, attempt to set up Bing tool
if bing_connection_name:
    try:
        # Retrieve Bing connection using project client
        bing_connection = project_client.connections.get(connection_name=bing_connection_name)
        conn_id = bing_connection.id
        
        # Initialize BingGroundingTool with the connection ID
        bing_tool = BingGroundingTool(connection_id=conn_id)
        print("bing > connected")
    except Exception as ex:
        # Handle any exceptions that occur during Bing tool setup
        print(f"bing > not connected: {ex}")

bing > connected


# Agent Configuration
Create or update the KYC agent with specific instructions and model deployment.

In [28]:
# Import required models and tools from azure.ai.projects
from azure.ai.projects.models import FunctionTool, ToolSet

# Import KYC functions
from kyc_functions import get_kyc_data, update_kyc_data

# Define agent name
AGENT_NAME = "kyc-agent"

# Search for existing agent with the specified name
found_agent = next(
    (a for a in project_client.agents.list_agents().data if a.name == AGENT_NAME),
    None
)

# Build toolset and add Bing tool if available
toolset = ToolSet()
if bing_tool:
    toolset.add(bing_tool)
toolset.add(FunctionTool({get_kyc_data, update_kyc_data}))

# Define agent instructions
instructions = """
You are a helpful KYC agent. For every question the user asks, you should:
1. Retrieve KYC data from Cosmos DB using get_kyc_data
2. Search via Bing search request to verify the data from the KYC database is up to date 

If you find any outdated or conflicting data, propose to update the KYC data using update_kyc_data. Before updating any KYC data, you should first confirm it with the user. Visualize what the change will be (Before and After values).

If the data seems to be up to date, provide detailed answers to the user. Always include the source of the information in the beginning of your answer (in quotes).
"""

# Create or update agent with the specified model, name, instructions, and toolset
if found_agent:
    agent = project_client.agents.update_agent(
        assistant_id=found_agent.id,
        model=found_agent.model,
        instructions=found_agent.instructions,
        toolset=toolset
    )
else:
    agent = project_client.agents.create_agent(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4"),
        name=AGENT_NAME,
        instructions=instructions,
        toolset=toolset
    )

# Toolset and Functions
Configure the toolset with Bing search and KYC functions for data retrieval and updates.

In [29]:
# Import required models and tools from azure.ai.projects
from azure.ai.projects.models import FunctionTool, ToolSet

# Import KYC functions
from kyc_functions import get_kyc_data, update_kyc_data

# Define agent name
AGENT_NAME = "kyc-agent"

# Search for existing agent with the specified name
found_agent = next(
    (a for a in project_client.agents.list_agents().data if a.name == AGENT_NAME),
    None
)

# Build toolset and add Bing tool if available
toolset = ToolSet()
if bing_tool:
    toolset.add(bing_tool)
toolset.add(FunctionTool({get_kyc_data, update_kyc_data}))

# Define agent instructions
instructions = """
You are a helpful KYC agent. For every question the user asks, you should:
1. Retrieve KYC data from Cosmos DB using get_kyc_data
2. Search via Bing search request to verify the data from the KYC database is up to date 

If you find any outdated or conflicting data, propose to update the KYC data using update_kyc_data. Before updating any KYC data, you should first confirm it with the user. Visualize what the change will be (Before and After values).

If the data seems to be up to date, provide detailed answers to the user. Always include the source of the information in the beginning of your answer (in quotes).
"""

# Create or update agent with the specified model, name, instructions, and toolset
if found_agent:
    agent = project_client.agents.update_agent(
        assistant_id=found_agent.id,
        model=found_agent.model,
        instructions=found_agent.instructions,
        toolset=toolset
    )
else:
    agent = project_client.agents.create_agent(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        name=AGENT_NAME,
        instructions=instructions,
        toolset=toolset
    )

# Gradio Interface Setup
Create interactive chat interface using Gradio with example questions and chat history management.

In [ ]:
# Import Gradio library for creating the chat interface
import gradio as gr

# Import the function to create the chat interface
from chat_ui import create_chat_interface

# Create a new thread for the chat
thread = project_client.agents.create_thread()

# Create the chat interface using the project client, agent, and thread
azure_kyc_chat = create_chat_interface(project_client, agent, thread)

# Define the Gradio Blocks interface
with gr.Blocks(title="Azure AI - FSI Agent Labs") as demo:
    gr.Markdown("## Azure AI - FSI Agent Lab with Cosmos DB + Bing")

    # Create a chatbot component
    chatbot = gr.Chatbot(type="messages")
    
    # Create a textbox for user input
    input_box = gr.Textbox(label="Ask the KYC agent...")

    # Function to clear chat history
    def clear_history():
        global thread
        thread = project_client.agents.create_thread()
        return []

    # Function to set example question
    def set_example_question(question):
        return question

    # Add a button to clear chat history
    with gr.Row():
        gr.Button("Clear Chat").click(fn=clear_history, outputs=chatbot)
    
    # Add example questions
    gr.Markdown("### Example Questions")
    with gr.Row():
        q1 = gr.Button("Please validate / update the PEP (Politically Exposed Person) status of Alain Berset based on what you find on the web.")
        q2 = gr.Button("Is Satya Nadella a customer?")

    # Set up example question buttons to populate the input box and trigger the chat
    for btn in [q1, q2]:
        btn.click(fn=set_example_question, inputs=btn, outputs=input_box) \
           .then(azure_kyc_chat, inputs=[input_box, chatbot], outputs=[chatbot, input_box]) \
           .then(lambda: "", outputs=input_box)

    # Set up the input box to submit the chat and clear the input box after submission
    input_box.submit(azure_kyc_chat, inputs=[input_box, chatbot], outputs=[chatbot, input_box]) \
             .then(lambda: "", outputs=input_box)

# Launch the Gradio interface with debugging enabled
demo.queue().launch(debug=True)